In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from os.path import dirname, join
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os


def detect_and_predict_mask(frame, faceNet, maskNet):
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
                                 (104.0, 177.0, 123.0))

    faceNet.setInput(blob)
    detections = faceNet.forward()
    print(detections.shape)

    faces = []
    locs = []
    preds = []

    # loop over the detections
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # convert it from BGR to RGB channel and ordering, resize
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            # bounding boxes to their respective lists
            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # only make a predictions if at least one face was detected
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        preds = maskNet.predict(faces, batch_size=32)
    return (locs, preds)

# load our serialized face detector model from disk
prototxtPath = r"deploy.protext"
weightsPath = r"res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the face mask detector model from disk
maskNet = load_model("mask_detector.model")

# initialize the video stream
print("Starting the CAMERA...")
vs = VideoStream(src=0).start()

# loop over the frames from the video stream
while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    for (box, pred) in zip(locs, preds):
        # unpack the bounding box and predictions
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred

        # draw bounding box and text
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

        # display the label and bounding box rectangle on the output
        cv2.putText(frame, label, (startX, startY - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 5)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()


Starting the CAMERA...
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
1/1 [==============================] - 2s 2s/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 65ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 54ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 59ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 60ms/step
(1, 1, 200, 7)
1/1 [========================

1/1 [==============================] - 0s 50ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 56ms/step
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
1/1 [==============================] - 0s 66ms/step
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 

1/1 [==============================] - 0s 93ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 66ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 49ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 113ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 73ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 74ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 71ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 73ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 71ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 71ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 73ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 73ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 77ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 74ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 73ms/step
(1, 1, 20

1/1 [==============================] - 0s 56ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 46ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 50ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 56ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 50ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 49ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 61ms/step
(1, 1, 200, 7)
(1, 1, 200, 7)
1/1 [==============================] - 0s 54ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 57ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 55ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 54ms/step
(1, 1, 200, 7)
1/1 [==============================] - 0s 81ms/step
(1, 1, 200, 7)
(1, 1, 200, 7)
1/1 [==============================] - 0s 49ms/step
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 20